In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
mouse = pd.read_csv('../input/Data_Cortex_Nuclear.csv')

# <a class="anchor" id="data_example">데이터예시</a>

In [ ]:
mouse.head()

In [ ]:
mouse.shape

In [ ]:
mouse['Behavior'].value_counts()

In [ ]:
# Check total missing values for each column

# mouse.isnull().sum()

In [ ]:
# Check missing values for a randomly selected protein 'DYRK1A_N'
mouse[mouse['DYRK1A_N'].isnull()]

# <a class="anchor" id="class8_frequency">8개클래스 빈도 분석</a>

In [ ]:
# Plot frequency of unique values for 'class'

mouse['class'].value_counts().plot(kind='bar')

plt.title('Class Frequency Table')
plt.xlabel('Class Type')
plt.ylabel('Counts')
plt.xticks(rotation='horizontal')
plt.show()

<a href="#DYRK1A_N_histogram">DYRK1A_N_histogram</a>

In [ ]:
# Histogram of protein expression level - DYRK1A

protein = mouse['DYRK1A_N'].dropna()
plt.hist(protein, bins=25, edgecolor='black')
plt.title('Expression Frequency Table')
plt.xlabel('Protein Expression Level')
plt.ylabel('Counts')
plt.show()

# <a class="anchor" id="DYRK1A_N_box_plot">8개클래스별 DYRK1A_N 박스플롯</a>

In [ ]:
# Boxplot of Protein Expression Level by Class

protein = mouse[['DYRK1A_N', 'class']].dropna()
sns.boxplot(x='class', y='DYRK1A_N', data=protein)
plt.show()

# <a class="anchor" id="anova">분산분석</a>

In [ ]:
# 세개 이상의 집단에 대한 평균 비교를 하기 위해 ANOVA (분산분석) 을 시행함

# ANOVA 사용하기 위한 가정
# 1. 독립성 : 자료의 추출은 독립적으로 이루어졌음 (무선표집을 하였다면 만족하는 것으로 여김)
# 2. 정규성: 조사하고자 하는 자료 (variable in question) 의 모집단 분포는 정규분포를 따름
# 3. 등분산성: 모든 집단의 모분산은 동일함

# 8개 클래스 (8개 집단) 에 대해 DYRK1A_N 단백질 발현정도의 평균을 비교한다

# 귀무가설: 모든 집단의 평균이 동일하다
# 대립가설: 적어도 한 집단의 평균이 다른 집단들과 다르다

protein = mouse[['DYRK1A_N', 'class']].dropna()

g1 = protein[(protein['class'] == 'c-CS-m')]['DYRK1A_N']
g2 = protein[(protein['class'] == 'c-SC-m')]['DYRK1A_N']
g3 = protein[(protein['class'] == 'c-CS-s')]['DYRK1A_N']
g4 = protein[(protein['class'] == 'c-SC-s')]['DYRK1A_N']
g5 = protein[(protein['class'] == 't-CS-m')]['DYRK1A_N']
g6 = protein[(protein['class'] == 't-SC-m')]['DYRK1A_N']
g7 = protein[(protein['class'] == 't-CS-s')]['DYRK1A_N']
g8 = protein[(protein['class'] == 't-SC-s')]['DYRK1A_N']

print(stats.f_oneway(g1, g2, g3, g4, g5, g6, g7, g8)) 

# <a class="anchor" id="anova_result">분산분석결과</a>

검정통계량값 statistic = 62.08, p-value = 5.92e-75  
p-value < 0.05 이므로, 유의수준 5%에서, 8개 집단의 평균에서 통계적으로 유의미한 차이가 있음  

ANOVA 는 다중의 집단의 평균에서 유의미한 차이가 있는지만 검증함  
구체적으로 어떤 수준(집단)이 차이가 있는지 확인하려면 사후분석(post hoc tests) 이 필요함  

유의미한 차이가 없는 경우에는 사후분석할 필요가 없음

# <a class="anchor" id="ttest_ind">독립표본 t검정 시행함</a>

In [ ]:
# 아래 두 쌍의 독립표본에 대하여, 독립표본 t검정 시행함

# 그룹1 : 정상, 컨텍스트-공포, memantine
# 그룹2 : 정상, 공포-컨텍스트, memantine

# 그룹5 : 실험군, 컨텍스트-공포, memantine
# 그룹7 : 실험군, 공포-컨텍스트, saline

print(stats.ttest_ind(g1, g2))
print(stats.ttest_ind(g5, g7))

p-value<0.05이므로, 유의수준 5%에서, 그룹1과 그룹2는 평균에서 유의미한 차이가 있음  
p-value<0.05이므로, 유의수준 5%에서, 그룹5와 그룹7은 평균에서 유의미한 차이가 있음

In [ ]:
# Data Cleaning
# Remove columns with NaN >= 10, then drop rows with missing data and categorical columns

print(mouse.shape)
mouse_new = mouse[mouse.columns[mouse.isnull().sum() < 10]].dropna().drop(['MouseID', 'Genotype', 'Treatment', 'Behavior'], axis=1)
print(mouse_new.shape)

# <a class="anchor" id="data_preparation">훈련데이터준비</a>

In [ ]:
# feature : 77종류 단백질 발현정도
# target : 클래스1 (실험군, 컨텍스트-공포, saline), 클래스1 (실험군, 공포-컨텍스트, saline)
# 사용 머신러닝 모델 : 서포트 벡터 머신
# 훈련 유형 : Logistric regression (or classification)

df = mouse_new[(mouse_new['class'] == 'c-CS-s') | (mouse_new['class'] == 'c-SC-s')].copy()
df.loc[:, 'class'].replace({'c-CS-s': 1, 'c-SC-s': 0}, inplace=True)
X = df.drop(['class'], axis=1)
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(X_train.head(3))

<a href="#train_and_validation">train_and_validation</a>

In [ ]:
svm = SVC(gamma='scale')
svm.fit(X_train, y_train)

print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))